In [91]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pulp

In [115]:
# Typy specjalistów i ich liczność
specialists = {'A': 17, 'B': 12, 'C': 8, 'D': 10}

# Rodzaje zadań i ich zapotrzebowanie
tasks = {'I': 10, 'II': 15, 'III': 12}

# Wydajność specjalistów
efficiency = {
    'A': {'I': 14.0, 'II': 8.0, 'III': 15.0},
    'B': {'I': 9.0, 'II': 9.0, 'III': 10.0},
    'C': {'I': 6.0, 'II': 7.0, 'III': 6.0},
    'D': {'I': 11.0, 'II': 4.0, 'III': 9.0}
}

# Koszty utrzymania specjalistów

costs = {
    'A': {'I': 12.0, 'II': 12.0, 'III': 18.0},
    'B': {'I': 8.0, 'II': 19.0, 'III': 14.0},
    'C': {'I': 10.0, 'II': 15.0, 'III': 11.0},
    'D': {'I': 12.0, 'II': 13.0, 'III': 14.0}
}


In [116]:
# Przygotowanie danych do wyświetlenia w tabelach

# Dane dotyczące specjalistów
specialists_data = {'Specialist Type': list(specialists.keys()), 'Availability': list(specialists.values())}
specialists_df = pd.DataFrame(specialists_data)

# Dane dotyczące zadań
tasks_data = {'Task Type': list(tasks.keys()), 'Demand': list(tasks.values())}
tasks_df = pd.DataFrame(tasks_data)

# Dane dotyczące wydajności
efficiency_data = []
for specialist, task_eff in efficiency.items():
    for task, eff in task_eff.items():
        efficiency_data.append({'Specialist': specialist, 'Task': task, 'Efficiency': eff})
efficiency_df = pd.DataFrame(efficiency_data)

# Dane dotyczące kosztów
costs_data = []
for specialist, task_costs in costs.items():
    for task, cost in task_costs.items():
        costs_data.append({'Specialist': specialist, 'Task': task, 'Cost': cost})
costs_df = pd.DataFrame(costs_data)

# Wyświetlenie DataFrame'ów
specialists_df, tasks_df, efficiency_df, costs_df

(  Specialist Type  Availability
 0               A            17
 1               B            12
 2               C             8
 3               D            10,
   Task Type  Demand
 0         I      10
 1        II      15
 2       III      12,
    Specialist Task  Efficiency
 0           A    I        14.0
 1           A   II         8.0
 2           A  III        15.0
 3           B    I         9.0
 4           B   II         9.0
 5           B  III        10.0
 6           C    I         6.0
 7           C   II         7.0
 8           C  III         6.0
 9           D    I        11.0
 10          D   II         4.0
 11          D  III         9.0,
    Specialist Task  Cost
 0           A    I  12.0
 1           A   II  12.0
 2           A  III  18.0
 3           B    I   8.0
 4           B   II  19.0
 5           B  III  14.0
 6           C    I  10.0
 7           C   II  15.0
 8           C  III  11.0
 9           D    I  12.0
 10          D   II  13.0
 11          D  III 

In [117]:
# Wagi dla celów optymalizacji (przykładowe wartości)
w_efficiency = 1.0  # Waga dla wydajności
w_cost = 0.01       # Waga dla kosztów, może wymagać dostosowania


In [118]:
# print((s, t) for s in specialists for t in tasks)

for s, t in ((s, t) for s in specialists for t in tasks):
    print(f"Specialist: {s}, Task: {t}")

Specialist: A, Task: I
Specialist: A, Task: II
Specialist: A, Task: III
Specialist: B, Task: I
Specialist: B, Task: II
Specialist: B, Task: III
Specialist: C, Task: I
Specialist: C, Task: II
Specialist: C, Task: III
Specialist: D, Task: I
Specialist: D, Task: II
Specialist: D, Task: III


In [119]:
# Inicjalizacja modelu
model = pulp.LpProblem("Optimal_Specialist_Allocation", pulp.LpMaximize)

# Zmienne decyzyjne
allocation = pulp.LpVariable.dicts("Allocation", 
                                   ((s, t) for s in specialists for t in tasks), lowBound=0, cat='Integer')


In [120]:
# Funkcja celu - zbilansowanie wydajności i kosztów
model += (w_efficiency * pulp.lpSum([efficiency[s][t] * allocation[(s, t)] for s in specialists for t in tasks]) -
         w_cost * pulp.lpSum([costs[s][t] * allocation[(s, t)] for s in specialists for t in tasks]))


In [121]:
# Ograniczenia dotyczące dostępności specjalistów
for s in specialists:
    model += pulp.lpSum([allocation[(s, t)] for t in tasks]) <= specialists[s]

# Ograniczenia dotyczące zapotrzebowania na zadania
for t in tasks:
    model += pulp.lpSum([allocation[(s, t)] for s in specialists]) >= tasks[t]


In [122]:
# Rozwiązanie problemu
model.solve()

1

In [123]:
# Wyświetlenie wyników
for v in model.variables():
    # if v.varValue > 0:
    print(v.name, "=", v.varValue)

Allocation_('A',_'I') = 0.0
Allocation_('A',_'II') = 0.0
Allocation_('A',_'III') = 17.0
Allocation_('B',_'I') = 0.0
Allocation_('B',_'II') = 7.0
Allocation_('B',_'III') = 5.0
Allocation_('C',_'I') = 0.0
Allocation_('C',_'II') = 8.0
Allocation_('C',_'III') = 0.0
Allocation_('D',_'I') = 10.0
Allocation_('D',_'II') = 0.0
Allocation_('D',_'III') = 0.0


In [125]:
# Tworzenie DataFrame do wyświetlenia wyników
results_df = pd.DataFrame(columns=["Specialist", "Task", "Allocation"])



In [126]:
for v in model.variables():
    # if v.varValue > 0:
    specialist, task = v.name.split("_")[1:]
    new_row = pd.DataFrame({"Specialist": [specialist], "Task": [task], "Allocation": [v.varValue]})
    results_df = pd.concat([results_df, new_row], ignore_index=False)


C:\Users\Piotr\AppData\Local\Temp\ipykernel_2648\2240924012.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=False)


In [127]:
# Wyświetlenie wyników
if model.status == pulp.LpStatusOptimal:
    print("Znaleziono optymalne rozwiązanie")
    display(results_df)
else:
    print("Nie znaleziono optymalnego rozwiązania")

Znaleziono optymalne rozwiązanie


,Specialist,Task,Allocation
0,"('A',",'I'),0.0
0,"('A',",'II'),0.0
0,"('A',",'III'),17.0
0,"('B',",'I'),0.0
0,"('B',",'II'),7.0
0,"('B',",'III'),5.0
0,"('C',",'I'),0.0
0,"('C',",'II'),8.0
0,"('C',",'III'),0.0
0,"('D',",'I'),10.0


In [128]:
def optimize_allocation(w_efficiency, w_cost):
    # Dane dotyczące specjalistów, zadań, wydajności i kosztów
    specialists = {'A': 17, 'B': 12, 'C': 8, 'D': 10}
    tasks = {'I': 10, 'II': 15, 'III': 12}
    efficiency = {
        'A': {'I': 1.2, 'II': 1.0, 'III': 0.8},
        'B': {'I': 0.9, 'II': 1.3, 'III': 0.7},
        'C': {'I': 0.7, 'II': 0.8, 'III': 1.1},
        'D': {'I': 1.0, 'II': 0.9, 'III': 1.2}
    }
    costs = {
        'A': {'I': 100, 'II': 120, 'III': 110},
        'B': {'I': 90, 'II': 100, 'III': 95},
        'C': {'I': 85, 'II': 105, 'III': 90},
        'D': {'I': 95, 'II': 115, 'III': 100}
    }

    # Tworzenie tabeli funkcji celu
    objective_data = []
    for s, tasks_eff in efficiency.items():
        for t, eff in tasks_eff.items():
            objective_data.append({
                "Variable": f"Allocation_{s}_{t}",
                "Coefficient (Efficiency)": eff * w_efficiency,
                "Coefficient (Cost)": -costs[s][t] * w_cost
            })
    objective_df = pd.DataFrame(objective_data)

    # Tworzenie tabeli ograniczeń
    constraints_data = []
    for s, avail in specialists.items():
        constraints_data.append({"Constraint": f"Sum of {s} allocations", "Value": avail})
    for t, demand in tasks.items():
        constraints_data.append({"Constraint": f"Total for task {t}", "Value": demand})
    constraints_df = pd.DataFrame(constraints_data)

    # Inicjalizacja modelu
    model = pulp.LpProblem("Optimal_Specialist_Allocation", pulp.LpMaximize)

    # Zmienne decyzyjne
    allocation = pulp.LpVariable.dicts("Allocation", 
                                       ((s, t) for s in specialists for t in tasks), 
                                       lowBound=0, cat='Integer')

    # Dodanie funkcji celu do modelu
    model += pulp.lpSum([efficiency[s][t] * w_efficiency * allocation[(s, t)] 
                         - costs[s][t] * w_cost * allocation[(s, t)] 
                         for s in specialists for t in tasks])

    # Dodanie ograniczeń dotyczących dostępności specjalistów
    for s in specialists:
        model += pulp.lpSum([allocation[(s, t)] for t in tasks]) <= specialists[s]

    # Dodanie ograniczeń dotyczących zapotrzebowania na zadania
    for t in tasks:
        model += pulp.lpSum([allocation[(s, t)] for s in specialists]) >= tasks[t]

    # Rozwiązanie problemu
    model.solve()

    # Tworzenie DataFrame do wyświetlenia wyników
    results_df = pd.DataFrame(columns=["Specialist", "Task", "Allocation"])
    for v in model.variables():
        if v.varValue > 0:
            specialist, task = v.name.split("_")[1:]
            new_row = pd.DataFrame({"Specialist": [specialist], "Task": [task], "Allocation": [v.varValue]})
            results_df = pd.concat([results_df, new_row], ignore_index=False)
            

    # Zwrócenie wyników i tablic
    return results_df, objective_df, constraints_df

Variable: Reprezentuje zmienną decyzyjną w modelu, która w tym przypadku określa przydział specjalisty do określonego zadania. Na przykład Allocation_A_I oznacza przydział specjalistów typu A do zadania I.

    Coefficient (Efficiency): Jest to współczynnik efektywności przypisany do konkretnej kombinacji specjalisty i zadania, pomnożony przez wagę efektywności (w_efficiency). Na przykład, jeśli efektywność specjalisty typu A przy zadaniu I wynosi 1.2, a waga efektywności wynosi 1.0, to współczynnik efektywności dla Allocation_A_I będzie wynosić 1.2×1.0=1.21.2×1.0=1.2.

    Coefficient (Cost): Jest to współczynnik kosztu przypisany do tej samej kombinacji specjalisty i zadania, pomnożony przez wagę kosztu (w_cost), ale z wartością ujemną, ponieważ chcemy minimalizować koszty. Na przykład, jeśli koszt utrzymania specjalisty typu A przy zadaniu I wynosi 100, a waga kosztu wynosi 0.01, to współczynnik kosztu dla Allocation_A_I będzie wynosić −100×0.01=−1.0−100×0.01=−1.0.

W funkcji celu modelu liniowego, te współczynniki są mnożnikami dla zmiennych decyzyjnych. Funkcja celu sumuje te iloczyny dla wszystkich kombinacji specjalistów i zadań. W ten sposób model stara się maksymalizować ogólną wydajność przy jednoczesnej minimalizacji kosztów.

In [129]:
w_efficiency = 1.0  # Waga dla wydajności
w_cost = 0.01       # Waga dla kosztów, może wymagać dostosowania
optimize_allocation(w_efficiency, w_cost)

C:\Users\Piotr\AppData\Local\Temp\ipykernel_2648\1400068516.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=False)


(  Specialist    Task  Allocation
 0      ('A',    'I')        14.0
 0      ('A',   'II')         3.0
 0      ('B',   'II')        12.0
 0      ('C',  'III')         8.0
 0      ('D',  'III')        10.0,
             Variable  Coefficient (Efficiency)  Coefficient (Cost)
 0     Allocation_A_I                       1.2               -1.00
 1    Allocation_A_II                       1.0               -1.20
 2   Allocation_A_III                       0.8               -1.10
 3     Allocation_B_I                       0.9               -0.90
 4    Allocation_B_II                       1.3               -1.00
 5   Allocation_B_III                       0.7               -0.95
 6     Allocation_C_I                       0.7               -0.85
 7    Allocation_C_II                       0.8               -1.05
 8   Allocation_C_III                       1.1               -0.90
 9     Allocation_D_I                       1.0               -0.95
 10   Allocation_D_II                       0.9

In [106]:
def optimize_allocation_with_sensitivity(w_efficiency, w_cost, cost_change_A_II, efficiency_change_A):
    # Aktualizacja danych dotyczących kosztów i wydajności
    costs['A']['II'] += cost_change_A_II
    for task in efficiency['A']:
        efficiency['A'][task] += efficiency_change_A

    # Reszta funkcji pozostaje bez zmian
    model = pulp.LpProblem("Optimal_Specialist_Allocation", pulp.LpMaximize)
    allocation = pulp.LpVariable.dicts("Allocation", 
                                       ((s, t) for s in specialists for t in tasks), 
                                       lowBound=0, cat='Integer')

    model += pulp.lpSum([efficiency[s][t] * w_efficiency * allocation[(s, t)] 
                         - costs[s][t] * w_cost * allocation[(s, t)] 
                         for s in specialists for t in tasks])

    for s in specialists:
        model += pulp.lpSum([allocation[(s, t)] for t in tasks]) <= specialists[s]
    for t in tasks:
        model += pulp.lpSum([allocation[(s, t)] for s in specialists]) >= tasks[t]

    model.solve()

    results_df = pd.DataFrame(columns=["Specialist", "Task", "Allocation"])
    for v in model.variables():
        if v.varValue > 0:
            specialist, task = v.name.split("_")[1:]
            new_row = pd.DataFrame({"Specialist": [specialist], "Task": [task], "Allocation": [v.varValue]})
            results_df = pd.concat([results_df, new_row], ignore_index=False)
            

    return results_df

# Przykładowe wywołanie funkcji z zmianą kosztów i wydajności
cost_change_A_II = 10  # Zwiększenie kosztów o 10
efficiency_change_A = 0.1  # Zwiększenie wydajności o 0.1
df = optimize_allocation_with_sensitivity(1.0, 0.01, cost_change_A_II, efficiency_change_A)
df


C:\Users\Piotr\AppData\Local\Temp\ipykernel_2648\69598858.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=False)


,Specialist,Task,Allocation
0,"('A',",'I'),17.0
0,"('B',",'II'),12.0
0,"('C',",'III'),8.0
0,"('D',",'II'),3.0
0,"('D',",'III'),7.0


In [107]:
print(result_df.iterrows)

<bound method DataFrame.iterrows of   Specialist    Task  Allocation
0      ('A',    'I')        17.0
1      ('B',    'I')         3.0
2      ('B',   'II')         9.0
3      ('C',  'III')         8.0
4      ('D',   'II')         6.0
5      ('D',  'III')         4.0>


In [108]:
import matplotlib.pyplot as plt
import numpy as np

def perform_sensitivity_analysis(cost_range, efficiency_range, w_efficiency, w_cost):
    cost_results = []
    efficiency_results = []

    # Analiza zmiany kosztów
    # for cost_change in cost_range:
    #     result_df = optimize_allocation_with_sensitivity(w_efficiency, w_cost, cost_change, 0)
    #     total_cost = sum([costs[row['Specialist']][row['Task']] * row['Allocation']
    #                       for index, row in result_df.iterrows()])
    #     cost_results.append(total_cost)


    for cost_change in cost_range:
        result_df = optimize_allocation_with_sensitivity(w_efficiency, w_cost, cost_change, 0)
        total_cost = sum([costs[row['Specialist']][row['Task']] * row['Allocation']
                        for index, row in result_df.iterrows()])
        cost_results.append(total_cost)

# Podobnie dla analizy zmiany wydajności




    # Analiza zmiany wydajności
    for efficiency_change in efficiency_range:
        result_df = optimize_allocation_with_sensitivity(w_efficiency, w_cost, 0, efficiency_change)
        total_efficiency = sum([efficiency[row['Specialist']][row['Task']] * row['Allocation']
                                for index, row in result_df.iterrows()])
        efficiency_results.append(total_efficiency)

    # Tworzenie wykresów
    plt.figure(figsize=(12, 6))

    # Wykres dla zmiany kosztów
    plt.subplot(1, 2, 1)
    plt.plot(cost_range, cost_results, marker='o')
    plt.title('Zmiana ogólnych kosztów w zależności od zmiany kosztu A_II')
    plt.xlabel('Zmiana kosztu A_II')
    plt.ylabel('Łączny koszt')

    # Wykres dla zmiany wydajności
    plt.subplot(1, 2, 2)
    plt.plot(efficiency_range, efficiency_results, marker='o', color='green')
    plt.title('Zmiana ogólnej wydajności w zależności od zmiany wydajności A')
    plt.xlabel('Zmiana wydajności A')
    plt.ylabel('Łączna wydajność')

    plt.tight_layout()
    plt.show()

# Zakresy zmian dla analizy
cost_range = np.linspace(-20, 20, 10)  # Zakres zmiany kosztów
efficiency_range = np.linspace(-0.5, 0.5, 10)  # Zakres zmiany wydajności

# Przeprowadzenie analizy
perform_sensitivity_analysis(cost_range, efficiency_range, 1.0, 0.01)


C:\Users\Piotr\AppData\Local\Temp\ipykernel_2648\69598858.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=False)


KeyError: "('A',"

In [85]:


# Zakres wag do eksploracji
w_efficiency_range = np.linspace(0.1, 1, 10)
w_cost_range = np.linspace(0.01, 0.1, 10)

pareto_data = []

for w_efficiency in w_efficiency_range:
    for w_cost in w_cost_range:
        result_df = optimize_allocation_with_sensitivity(w_efficiency, w_cost, 0, 0)
        total_cost = sum([costs[row['Specialist']][row['Task']] * row['Allocation']
                          for index, row in result_df.iterrows()])
        total_efficiency = sum([efficiency[row['Specialist']][row['Task']] * row['Allocation']
                                for index, row in result_df.iterrows()])
        pareto_data.append((total_efficiency, total_cost))

# Tworzenie DataFrame i wykresu
pareto_df = pd.DataFrame(pareto_data, columns=['Total Efficiency', 'Total Cost'])
pareto_df = pareto_df.drop_duplicates().sort_values(by='Total Efficiency')

plt.figure(figsize=(10, 6))
plt.scatter(pareto_df['Total Efficiency'], pareto_df['Total Cost'], color='blue')
plt.title('Wykres Pareto dla Wydajności i Kosztów')
plt.xlabel('Łączna Wydajność')
plt.ylabel('Łączny Koszt')
plt.grid(True)
plt.show()

pareto_df


C:\Users\Piotr\AppData\Local\Temp\ipykernel_2648\2734447012.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


KeyError: "('A',"